# CAPSTONE - Grade Predictions for A Level H2 Math
IMPORTANT NOTE: You may run this notebook but the results will change as only 500 rows of data for each batch is shared due to confidentiality. In addition, Cell 6 will not run properly without inputting the numbers of each grade, but we cannot disclose these numbers. You may enter fictitious data on your own to see how the code works.

---
# (1) Benchmark Scores
A complex model may not be the best solution as ease of implementation is a prime consideration. One easily implementable way for generating predictions is to use the Prelim results for the batch and an average of a few years of past year A level grades to predict how well a student of that particular school will do. 

In this notebook, we will use only the Prelim marks for H2 Math and 3 years' past year A level grades to make the predictions and evaluate them using the tools described in the [Introduction](0_Introduction.ipynb) notebook.

## Contents
- [Method of Prediction](#Method-of-Prediction)
- [Preparing Data](#Preparing-Data)
- [Predictions](#Predictions)
- [Evaluation](#Evaluation)
- [Conclusion](#Conclusion)

---

# Method of Prediction

We generate the predictions by sorting the Math Prelim marks in descending order, predicting grade A for the top a% based on the percentage of A grades in the past 3 years, predicting grade B for the next b% and so on. As identical marks should get the same prediction, we will bump any prediction at a grade boundary up to the better grade.

In order to capture all students in the lowest grade category, we look at the highest mark achieved by a student in this category and predict the lowest grade category for all students with the same mark or lower.

For students who did not take Prelims (marks recorded as 'MC' or 'EX'), past experiences tell us that students who missed Prelims may get any grade from A to U. It will be unfair to the students to simply predict a middle grade like C, so the school will have to find another way to make a prediction, such as getting the subject tutor to make a prediction based on his understanding of the student's ability. For the purpose of benchmarking, we will remove these students who did not take Prelims, and will later attempt to make a grade prediction using machine learning models.

---

# Preparing Data

In [1]:
# imports
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, recall_score, confusion_matrix

# disable warning 'A value is trying to be set on a copy of a slice from a DataFrame'
pd.options.mode.chained_assignment = None

In [2]:
# load data
# feature MA2 Prelim marks
X_18 = pd.read_csv('../data_raw/18 Batch Prelim Marks.csv', usecols=['NEWID','MA2'], index_col='NEWID')
X_19 = pd.read_csv('../data_raw/19 Batch Prelim Marks.csv', usecols=['NEWID','MA2'], index_col='NEWID')
X_20 = pd.read_csv('../data_raw/20 Batch Prelim Marks.csv', usecols=['NEWID','MA2'], index_col='NEWID')
# target A level grades
y_18 = pd.read_csv('../data_raw/18 Batch GCEA.csv', usecols=['NEWID','MA2'], index_col='NEWID')
y_19 = pd.read_csv('../data_raw/19 Batch GCEA.csv', usecols=['NEWID','MA2'], index_col='NEWID')
y_20 = pd.read_csv('../data_raw/20 Batch GCEA.csv', usecols=['NEWID','MA2'], index_col='NEWID')

# lists for looping
X = [X_18, X_19, X_20]
y = [y_18, y_19, y_20]
batch = [18, 19, 20]

for i in range(3):
    # drop rows without MA2 Prelim marks
    X[i].dropna(inplace=True)
    
#HIDDEN due to confidentiality
#     # print number of rows
#     print(f'No. of rows: {X[i].shape[0]}') 
    # display rows with non-numeric values
    display(X[i][pd.to_numeric(X[i]['MA2'], errors='coerce').isna()])
    print(f'Batch {batch[i]}\n')

,MA2
NEWID,
20180182,MC
20180196,MC
20180409,MC
20181137,MC


Batch 18



,MA2
NEWID,
20190504,MC


Batch 19



,MA2
NEWID,
20200346,EX
20200402,MC
20200485,MC
20200583,EX
20200696,MC


Batch 20



4, 1 and 5 students did not have Math Prelim marks in batch 18, 19 and 20. We will drop these students and have their A level grade predicted by their Math tutor.

In [3]:
# drop rows that are 'MC' or 'EX'
for i in range(3):
    X[i] = X[i].loc[~X[i]['MA2'].isin(['MC', 'EX'])]
    X[i]['MA2'] = X[i]['MA2'].astype('float')
    # sort values so that predictions can be input by ranking
    X[i].sort_values('MA2', ascending=False, inplace=True)

In [4]:
# drop rows without MA2 A level grade
for i in range(3):
    y[i].dropna(inplace=True)

#HIDDEN due to confidentiality
#     # print number of rows and value counts
#     print(f'Batch {batch[i]}: {y[i].shape[0]} rows')
#     display(y[i].value_counts('MA2').sort_index())
#     print()

Note that 3 students in batch 20 with Prelim data did not receive an A level grade in the end. Most likely there might have been similar students in batch 18 and batch 19 but these were removed from the school database before the data was given to us.

In [5]:
# map grades to numbers, merging ESU into one category since frequency is low ( < 5)
grade_dict_target = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'S': 4, 'U': 4}
for i in range(3):
    y[i]['MA2'].replace(grade_dict_target, inplace=True)

# export to csv for modelling
y[0].to_csv('../data_cleaned/y_18.csv')
y[1].to_csv('../data_cleaned/y_19.csv')
y[2].to_csv('../data_cleaned/y_20.csv')

---
# Predictions

## Create predictions using past 3 years' results

In [6]:
# 3 years A level grades from batch 17, 18 and 19

#HIDDEN history due to confidentiality
# note that this cell was run with actual numbers in the line commented out below
# history = {0: num_A, 1: num_B, 2: num_C, 3: num_D, 4: num_ESU}
df = pd.DataFrame(history.values(), index=history.keys(), columns=['num_students'])

num_pred_18 = round(df['num_students']/df['num_students'].sum() * X[0].shape[0],0).astype('int')
num_pred_19 = round(df['num_students']/df['num_students'].sum() * X[1].shape[0],0).astype('int')
num_pred_20 = round(df['num_students']/df['num_students'].sum() * X[2].shape[0],0).astype('int')

# list for looping
num_pred = [num_pred_18, num_pred_19, num_pred_20]

for i in range(3):
    # adjust by adding/removing from grade A if total does not tally
    num_pred[i][0] -= num_pred[i].sum() - X[i].shape[0]

#HIDDEN due to confidentiality
#     # print predictions
#     print(f' Batch {batch[i]} '.center(36, "="))
#     print(num_pred[i])
#     print()

## Adjust for students with same marks to get same grade

In [7]:
for i in range(3):
    # create predictions column
    X[i]['Pred'] = np.nan
    pred = []
    for g in range(5):
        pred += [g] * num_pred[i][g]
    X[i]['Pred'] = np.array(pred)
    
    # students with same marks for Prelim should get same prediction
    # replace prediction with better grade if same marks occur at grade boundaries
    print(f' Batch {batch[i]} '.center(36, "="))
    for g in range(4):
        print(' Before adjusting '.center(24, "="))    
        ind = num_pred[i].cumsum().iloc[g]
        print(X[i].iloc[ind-1:ind+1])
        print()
        
        # check grade boundaries after adjusting - same score should have same prediction
        print(' After adjusting '.center(24, "="))
        if X[i].iloc[ind-1]['MA2'] == X[i].iloc[ind]['MA2']:
            X[i].loc[X[i]['MA2'] == X[i].iloc[ind]['MA2'], 'Pred'] = i+1
        print(X[i].iloc[ind-2:ind+2])
        print()

============= Batch 18 =============
=== Before adjusting ===
           MA2  Pred
NEWID               
20180881  51.5     0
20180558  51.5     1

=== After adjusting ====
           MA2  Pred
NEWID               
20180165  51.5     1
20180881  51.5     1
20180558  51.5     1
20180150  51.5     1

=== Before adjusting ===
           MA2  Pred
NEWID               
20180806  42.0     1
20180453  42.0     2

=== After adjusting ====
           MA2  Pred
NEWID               
20180729  42.0     1
20180806  42.0     1
20180453  42.0     1
20181119  41.5     2

=== Before adjusting ===
           MA2  Pred
NEWID               
20180691  33.5     2
20181145  33.5     3

=== After adjusting ====
           MA2  Pred
NEWID               
20180267  34.0     2
20180691  33.5     1
20181145  33.5     1
20180977  33.0     3

=== Before adjusting ===
           MA2  Pred
NEWID               
20180929  27.0     3
20180397  25.5     4

=== After adjusting ====
           MA2  Pred
NEWID               


In [8]:
# create Benchmark dataframes
Benchmark_18 = pd.DataFrame()
Benchmark_19 = pd.DataFrame()
Benchmark_20 = pd.DataFrame()
# list for looping
Benchmark = [Benchmark_18, Benchmark_19, Benchmark_20]

for i in range(3):
    # join to target y
    Benchmark[i] = X[i].join(y[i], rsuffix='_target', how='inner')

#HIDDEN due to confidentiality
#     # check shape
#     print(f'Shape: {Benchmark[i].shape}')
    display(Benchmark[i].head(2))
    print(f'Batch {batch[i]}\n')

,MA2,Pred,MA2_target
NEWID,,,
20180265,97.0,0,0
20181075,94.5,0,0


Batch 18



,MA2,Pred,MA2_target
NEWID,,,
20190042,95.0,0,0
20191016,94.5,0,0


Batch 19



,MA2,Pred,MA2_target
NEWID,,,
20200811,97.4,0,0
20200415,96.8,0,0


Batch 20



## Capturing 100% 'ESU'

In [9]:
# highest Math Prelim marks for students in 'ESU'
for i in range(3):
    display(Benchmark[i][Benchmark[i]['MA2_target']==4].head(1))
    print(f'Batch {batch[i]}')

,MA2,Pred,MA2_target
NEWID,,,
20180195,29.5,3,4


Batch 18


,MA2,Pred,MA2_target
NEWID,,,
20190977,28.0,2,4


Batch 19


,MA2,Pred,MA2_target
NEWID,,,
20200727,27.7,2,4


Batch 20


The highest Math Prelim Mark in both batches is 29.5. We look for the total number of students who scored less than 30 in each batch. If we predicted all these students as 'ESU', we would get a 100% Recall for ESU.

In [10]:
for i in range(3):
    # adjust all predictions for score below 30 to 'ESU'
    Benchmark[i].loc[Benchmark[i]['MA2'] < 30, 'Pred'] = 4

    print(f' Batch {batch[i]} '.center(36, "="))
    # number of students with less than 30 marks in Math Prelims
    print(f"Number of students: {Benchmark[i][Benchmark[i]['MA2'] < 30].shape[0]}")
    # actual grades achieved by these students
    print(f"Actual grades: {sorted(Benchmark[i][Benchmark[i]['MA2'] < 30]['MA2_target'].unique())}")
    print()

============= Batch 18 =============
Number of students: 19
Actual grades: [1, 2, 3, 4]

============= Batch 19 =============
Number of students: 38
Actual grades: [1, 2, 3, 4]

============= Batch 20 =============
Number of students: 35
Actual grades: [0, 1, 2, 3, 4]



Some students who scored less than 30 marks in Math Prelims could get as high as 'A' in A levels.

---
# Evaluation

In [11]:
# define function to evaluate predictions
def get_scores(y_true, y_pred, batch):
    cohen_kappa = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    ESU_recall = recall_score(y_true, y_pred, average=None)[-1]
    macro_recall = recall_score(y_true, y_pred, average='macro')
    num_ESU = sum(1 if pred == 4  else 0 for pred in y_pred)
    
    df = pd.DataFrame([np.round([cohen_kappa, ESU_recall, macro_recall, num_ESU], 4)],
                      columns=["Weighted Cohen's Kappa", 
                               "Recall for ESU", 
                               "Macro Average Recall",
                               "Number Predicted as ESU"],
                      index = [batch]
                     )
    return df

# eval_pred NOT USED due to confidentiality
def eval_pred(y_true, y_pred, batch):
    get_scores(y_true, y_pred, batch)
    cm = pd.DataFrame(confusion_matrix(y_true, y_pred),
                      columns=['A','B','C','D','ESU'], 
                      index=['A','B','C','D','ESU'])
    cm.index.rename('Actual\Predicted',inplace=True)
    display(cm)
    print()
    display(pd.DataFrame([y_true.value_counts().sort_index(),
                          pd.Series(y_pred,name='Prediction').value_counts().sort_index()],
                         dtype='int'
                        ).rename(columns={0:'A', 1:'B', 2:'C', 3:'D', 4:'ESU'}))

In [12]:
# get_scores used instead of eval_pred due to confidentiality
df = pd.DataFrame(columns=["Weighted Cohen's Kappa", "Recall for ESU", "Macro Average Recall"])
for i in range(3):
    df_1 = get_scores(Benchmark[i]['MA2_target'], Benchmark[i]['Pred'], batch[i])
    df = pd.concat([df, df_1])
    
display(df)

,Weighted Cohen's Kappa,Recall for ESU,Macro Average Recall,Number Predicted as ESU
18,0.7581,1.0,0.5729,19.0
19,0.7204,1.0,0.5099,38.0
20,0.7768,1.0,0.5482,35.0


---
# Conclusion
The benchmark scores are as seen in the above table.

Results for batch 18 are better than for batch 19 as the proportion of A grades is much closer to the 3 years average that we used to generate the predictions.

The weighted kappa scores are quite high at more than 0.7, indicating that the benchmark method is quite successful. Since it can be easily implemented without using coding, it is a possible solution that schools can choose to adopt. Of course, the success of this benchmark method also depends on the consistency in teaching and setting of exam questions, which will affect the Prelim marks and final A level grades, so the schools must work to maintain a high standard with consistency, and it is best to take the rolling average of a few recent years' A level results.

We will move on to [Data Cleaning and EDA](2_Data_Cleaning_and_EDA.ipynb) to prepare the data for modelling before going on to [Modelling](3_Modelling.ipynb) where we will finally compare the confusion matrices of the benchmark method with the final production model.